In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
import requests

In [2]:
driver=webdriver.Chrome(executable_path="F:\Web_Scraping\chromedriver.exe")

In [3]:
url = "https://www.naukri.com/"

In [4]:
def get_url(post,location):
    template="https://in.indeed.com/jobs?q={}&l={}"
    url=template.format(post,location)
    return url

In [5]:
url=get_url('hadoop','Gurgaon')

In [6]:
url

'https://in.indeed.com/jobs?q=hadoop&l=Gurgaon'

In [14]:
#rextract Raw Html

In [15]:
response=requests.get(url)

In [16]:
response

<Response [200]>

In [17]:
response.reason

'OK'

In [18]:
soup=BeautifulSoup(response.text,'html.parser')

In [19]:
cards =soup.find_all('div','jobsearch-SerpJobCard')

In [20]:
print(len(cards))

15



## Prototype Cards of job search
 ##### first indexding of job card

In [21]:
card=cards[0]

In [22]:
atag=card.h2.a

In [23]:
job_title=atag.get('title')

In [24]:
job_url="https://in.indeed.com/" + atag.get('href')

In [25]:
comapny=card.find('span','company').text.strip()
comapny

'JANGOO TECHNOLOGIES'

In [26]:
job_location=card.find('div','recJobLoc').get('data-rc-loc')
job_location

'Delhi, Delhi'

In [27]:
job_summry=card.find('div','summary').text.strip()
job_summry

'Interested candidates may send their updated resume.\nThis job will on third party payroll with NTRO Delhi location near Noida.*.'

In [28]:
job_post=card.find('span','date').text
job_post

'13 days ago'

In [29]:
f_date=datetime.today().strftime('%d-%m-%y')
f_date

'14-11-20'

In [30]:
#use try function sometime salary is there is sometime not 
try:
    job_salary=card.find('span','salaryText').text.strip()
except AttributeError:
    job_salary=''

job_salary

'₹7,00,000 - ₹8,00,000 a year'

# Gernalize all model with function

In [31]:
def get_record(card):
    
    atag=card.h2.a
    job_title=atag.get('title')
    job_url="https://in.indeed.com/" + atag.get('href')
    comapny=card.find('span','company').text.strip()
    job_location=card.find('div','recJobLoc').get('data-rc-loc')
    job_summry=card.find('div','summary').text.strip()
    job_post=card.find('span','date').text
    f_date=datetime.today().strftime('%d-%m-%y')
    try:
        job_salary=card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary=''
    record = (job_title,comapny,job_location,job_summry,job_post,f_date,job_salary,job_url)
        
    return record

    

In [33]:
records=list()
for card in cards:
    record=get_record(card)
    if record:
        records.append(record)

In [34]:
records[0]

('ETL Developer',
 'JANGOO TECHNOLOGIES',
 'Delhi, Delhi',
 'Interested candidates may send their updated resume.\nThis job will on third party payroll with NTRO Delhi location near Noida.*.',
 '13 days ago',
 '14-11-20',
 '₹7,00,000 - ₹8,00,000 a year',
 'https://in.indeed.com//company/Jangoo-Technologies/jobs/ETL-Developer-3f8ef604f1d65f12?fccid=194bb6581e6d5c64&vjs=3')

In [35]:
while True:
    try:
        url="https://in.indeed.com/" + soup.find('a',{'arial-label':'Next'}).get('href')
    except AttributeError:
        break
    response=request.get(url)
    
    soup=BeautifulSoup(response.text,'html.preser')
    card =soup.find_all('div','jobsearch-SerpJobCard')
    
    for card in cards:
        record=get_record(card)
        records.append(record)



In [36]:
#all together
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
import requests

def get_url(post,location):
    template="https://in.indeed.com/jobs?q={}&l={}"
    url=template.format(post,location)
    return url

def get_record(card):
    
    atag=card.h2.a
    job_title=atag.get('title')
    job_url="https://in.indeed.com/" + atag.get('href')
    comapny=card.find('span','company').text.strip()
    job_location=card.find('div','recJobLoc').get('data-rc-loc')
    job_summry=card.find('div','summary').text.strip()
    job_post=card.find('span','date').text
    f_date=datetime.today().strftime('%d-%m-%y')
    try:
        job_salary=card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary=''
    record = (job_title,comapny,job_location,job_summry,job_post,f_date,job_salary,job_url)
        
    return record



def main(post,location):
    records=list()
    url=get_url(post,location)
    #save data as csv file
    
    while True:        
        response=requests.get(url)
        soup=BeautifulSoup(response.text,'html.parser')
        cards =soup.find_all('div','jobsearch-SerpJobCard')
    
        for card in cards:
            record=get_record(card)
            records.append(record)
        try:
            url="https://in.indeed.com/" + soup.find('a',{'arial-label':'Next'}).get('href')
        except AttributeError:
            break
        
    with open('Indeed_jobs.csv','w',newline='',encoding='utf-8')as f:
            writer=csv.writer(f)
            writer.writerow(['job_title','comapny','job_location','f_date','job_salary','job_url'])
            writer.writerows(records)


        
    
    

In [37]:
main('hadoop','Gurgaon')